## Inspired by statistics guru Hans Rosling who used an amazing new presentation tool in his [presentation](https://www.youtube.com/watch?v=hVimVzgtD6w), an interactive visualization showing the evolution of life expectancy in each country on four continents based on GDPPercap. 

In [1]:
import pandas as pd
import os

from bokeh.models import (LinearInterpolator,
                          CategoricalColorMapper,
                          ColumnDataSource,
                          HoverTool,
                          NumeralTickFormatter)

from bokeh.palettes import Spectral5
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure

from ipywidgets import interact
from IPython.display import IFrame

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data = pd.read_csv(os.path.join(dirname, filename), sep='\t', index_col = 'year')

output_notebook()

Loading BokehJS ...

__Data Colums__

In [2]:
print(data.dtypes)
data.head()

country       object
continent     object
lifeExp      float64
pop            int64
gdpPercap    float64
dtype: object


,country,continent,lifeExp,pop,gdpPercap
year,,,,,
1952,Afghanistan,Asia,28.801,8425333,779.445314
1957,Afghanistan,Asia,30.332,9240934,820.853030
1962,Afghanistan,Asia,31.997,10267083,853.100710
1967,Afghanistan,Asia,34.020,11537966,836.197138
1972,Afghanistan,Asia,36.088,13079460,739.981106


### Everything in *Bokeh* is a *ColumnDataSource* that accept two type of objects:
   - Data Frame
   - Python Dictionary
In this example *ColumnDataSource* will be fed the data yearly.

As not all countries have the same population, it makes sense to map the size of each point to the population size of the country. Therefore, we have used an interpolation method, which allowed us to map smaller (resp. larger) populations up to size 10 (resp. 60), and then linearly interpolated all countries in-between.

# HoverTool
Hovertool can be very customized and have a very specific HTMl (for example showing country flag), however in this case we have restricted usage to only show the country when we hover over a data point.


# Interactions
Using interactive widgets, we build a slider to update data, by slicing the data every year and updating the plot. In order to do this, we need to update the data in the *ColumnDataSource*, using the update() function. 

In [3]:

source = ColumnDataSource(data.loc[data.index.min()])

PLOT_OPTS = dict(
    height=400, width = 800, x_axis_type='log',
    x_range = (100,100000), y_range = (0,100),
    background_fill_color = 'black'
)

# Making Hover
hover = HoverTool(tooltips = '@country', show_arrow = False)


fig = figure(tools = [hover],
             toolbar_location = 'above',
             **PLOT_OPTS)

def update(year):
    """ Build New Data Based On The Year, method that update the source object """
    try:
        new_data =data.loc[year]
        source.data = new_data
    except KeyError:
        new_data = dict()
    fig.title.text = str(year)
    push_notebook()


# Interactive Widget
interact(update, year =(data.index.min(), data.index.max(),1))         

# Mapping biggest population to size 50
# Linearly interpolating all of the one in between
size_mapper = LinearInterpolator(
    x = [data['pop'].min(), data['pop'].max()],
    y = [10,60]
)

color_mapper = CategoricalColorMapper(
    factors = data.continent.unique().tolist(),
    palette = Spectral5,
)



fig.circle('gdpPercap', 'lifeExp',
           size = {'field':'pop', 'transform':size_mapper},
           color = {'field': 'continent', 'transform': color_mapper},
           alpha = 0.6,
           legend_field = 'continent',
           source = source,
           hover_color='white',
           line_color="white"
           )

# Move Legends off Canvas
fig.legend.border_line_color = None
fig.legend.location = (0,200)
fig.right.append(fig.legend[0])

fig.axis[0].formatter = NumeralTickFormatter(format = "$0")
show(fig, notebook_handle = True)

/opt/conda/lib/python3.7/site-packages/bokeh/io/notebook.py:252: UserWarning: Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()
  warn("Cannot find a last shown plot to update. Call output_notebook() and show(..., notebook_handle=True) before push_notebook()")
